Avaliação de Churn
O objetivo principal deste estudo é avaliar a probabilidade de churn dos clientes de um serviço de streaming nos próximos três meses, com a intenção de implementar políticas e ações que evitem a concretização desse cenário.

O problema está dividido em duas partes:

Previsão de Churn: Análise da probabilidade de evasão de clientes.
Análise Não Supervisionada: Estudo do perfil comportamental dos clientes.

In [ ]:
# Importações necessárias
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch

##Dask
import json
from dask import bag as db
from dask import dataframe as dd

# Importações de modelos e métricas do Scikit-Learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (mean_squared_error, confusion_matrix, precision_score, 
                             recall_score, f1_score, roc_curve, auc, accuracy_score, 
                             silhouette_score, adjusted_rand_score)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

# Algoritmos de machine learning
from sklearn import ensemble, linear_model, neighbors, tree, naive_bayes, svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering

# Configuração do pandas para exibição de colunas
pd.set_option('display.max_columns', 500)

# Supressão de warnings
import warnings
warnings.filterwarnings("ignore")

# Leitura dos dados
members = pq.read_table('/kaggle/input/case-data-master-2024/members.parquet')
transactions = pq.read_table('/kaggle/input/case-data-master-2024/transactions.parquet')
logs = pq.read_table('/kaggle/input/case-data-master-2024/user_logs.parquet')

# Conversão para DataFrames pandas
members_pd = members.to_pandas()
transactions_pd = transactions.to_pandas()
logs_pd = logs.to_pandas()

# Liberação de memória
del members
del transactions
del logs

1. Contexto
   
Neste estudo, daremos sequência à análise da evasão de clientes (churn) através de um modelo de classificação.

A definição do alvo (target) é guiada por alguns aspectos-chave: o cliente está inativo? Cancelou seu plano? Não ouviu música nos últimos três meses?

Para prever a saída de um cliente, é crucial observar a relação entre diferentes variáveis. Para isso, a avaliação de diversos cenários e hipóteses se torna necessária, algumas delas incluem:

Comportamento histórico do cliente: Qual foi o valor da sua assinatura e quanto música ele ouviu nos meses anteriores? Existe uma correlação com a possibilidade de churn?
Características socioeconômicas: Idade, gênero, cidade e canal de aquisição influenciam a taxa de churn?
Relação entre a quantidade ouvida no mês anterior e o churn: Existe uma conexão significativa?
Dias desde o registro: Clientes mais novos apresentam maior propensão ao churn?
Além de testar essas hipóteses, é fundamental compreender como os dados se comportam e se relacionam com o nosso alvo, mantendo apenas as variáveis que são realmente importantes e que influenciam o problema em estudo.

Dessa forma, dividiremos nosso processo em dois capítulos, cada um contendo as seguintes subfases:

Modelo de Previsão de Churn
Análise superficial dos dados
Análise exploratória
Definição das variáveis para treinamento
Construção do alvo (target)
Seleção de Features
Divisão em conjuntos de treinamento e teste
Treinamento do algoritmo
Hiperparametrização
Previsão do futuro e conclusão
Conclusão
Análise Não Supervisionada
Normalização e PCA (Análise de Componentes Principais)
Amostragem
K-means
Conclusão

A. Análise Superficial dos Dados
Members

Cálculo do Tempo desde o Cadastro: Podemos desenvolver uma função que calcule o tempo decorrido desde o cadastro de cada cliente.
Variáveis Categóricas:
A variável "cidade" apresenta 21 entradas distintas.
A variável "gênero" possui 3 categorias.
A variável "registered_via" contém 17 categorias. Esses dados possibilitam a construção de um one-hot encoding para melhor representação das variáveis categóricas no modelo.
Distribuição das Idades: A maior parte das idades dos clientes está entre 0 e 55 anos. O valor 0 provavelmente indica a falta de informação ou dados ausentes.
Tipos de Dados: Atualmente, todos os dados estão com o tipo dtype como object. Portanto, será necessário realizar um trabalho de conversão de tipos (casting) para adequar os dados aos tipos corretos.
Base de Clientes Ativos: Na última safra disponível (201612), contabilizamos aproximadamente 1 milhão de clientes ativos, com um número similar na penúltima safra.
Utilização da Base como Painel: Levaremos esta base como painel, pois as informações contidas são "estáticas" em relação ao cliente. Não é necessário manter o histórico dessas informações.

In [ ]:
# Visualização inicial das primeiras linhas e tipos de dados
print(members_pd.head())
print(members_pd.dtypes)

In [ ]:
# Exibição de contagem de valores únicos por coluna
for i in members_pd.columns:
    print(i)
    print(len(members_pd[i].unique()))
    print("")

In [ ]:
# Convertendo a coluna 'bd' para numérico, forçando erros para NaN
members_pd['bd'] = pd.to_numeric(members_pd['bd'], errors='coerce')

# Removendo idades inválidas e substituindo-as por NaN
members_pd['bd'] = members_pd['bd'].apply(lambda x: x if 0 < x < 100 else np.nan)

# Transformação da coluna de registro de data para tipo datetime
members_pd['registration_init_time'] = pd.to_datetime(members_pd['registration_init_time'], format='%Y%m%d')

# Calculando o tempo desde o cadastro em dias
last_date = pd.to_datetime('2017-03-31')  # Última data conhecida nos dados
members_pd['days_since_registration'] = (last_date - members_pd['registration_init_time']).dt.days

# Visualizando as mudanças e o tratamento inicial de outliers
print(members_pd[['city', 'registered_via', 'bd', 'days_since_registration']].head())

###Transactions

Construção da Target: Precisaremos da tabela de transações para construir a variável alvo (target) do nosso modelo.
Conversão de Tipos (Casting): Assim como em outras tabelas, será necessário realizar a conversão de tipos (casting) para garantir que os dados estejam adequadamente formatados.
Diferenciação de Clientes: É importante investigar a diferença entre clientes que pagam mensalmente e aqueles que optam pelo pagamento anual. Podemos modelar essa diferença, sendo mais prático dividir o preço pelo número de dias e normalizá-lo para um período padrão de 30 dias.
Variáveis Booleanas e Categóricas:
Temos algumas variáveis booleanas, como is_auto_renew e is_cancel, que indicam se a assinatura é renovada automaticamente e se foi cancelada, respectivamente.
Além disso, algumas variáveis podem ser tratadas como categóricas, como payment_method_id, que representa o método de pagamento utilizado.
Inatividade após Cancelamento: Se um cliente cancela a assinatura em um determinado mês, ele aparecerá como inativo no mês seguinte.
Usuários Inativos: É importante notar que os usuários inativos não são registrados na base de pagamentos, o que pode influenciar nossas análises.
Tratamento de Dados Ausentes: Observamos poucos casos em que o período do plano é 0 na base de dados. Podemos utilizar uma técnica de preenchimento de dados ausentes (fillna) para lidar com essas situações.

In [ ]:
# Visualizando as primeiras linhas e checando tipos de dados
transactions_pd.head()

In [ ]:
#Checando tipos de dados
transactions_pd.dtypes

In [ ]:
for i in transactions_pd.columns:
    print(i)
    print(len(transactions_pd[i].unique()))
    print("")

Informação de Dias da transação até expiração da conta

In [ ]:
# Convertendo as colunas para o tipo datetime
transactions_pd.transaction_date = pd.to_datetime(transactions_pd.transaction_date, format='%Y%m%d', errors='coerce')
transactions_pd.membership_expire_date = pd.to_datetime(transactions_pd.membership_expire_date, format='%Y%m%d', errors='coerce')

# Calculando os dias até a expiração da assinatura
transactions_pd['transaction_to_expire_days'] = (transactions_pd.membership_expire_date - transactions_pd.transaction_date).dt.days

# Verificando as primeiras linhas após a adição da nova coluna
print(transactions_pd[['transaction_date', 'membership_expire_date', 'transaction_to_expire_days']].head())


### Log de audições

Simplicidade dos Dados: Os dados são em sua maioria simples e numéricos. A coluna total_secs representa a quantidade total de tempo que o cliente ouviu músicas durante o mês, sendo uma medida não cumulativa.
Validação dos Dados: Não identificamos valores negativos nas variáveis, exceto na coluna total_secs, que necessitará de um tratamento específico para lidar com essas ocorrências atípicas.
Uso do Histórico: Dispomos de um histórico robusto de dados, permitindo a exploração de múltiplos meses para compreender a média histórica de consumo dos clientes. Isso possibilita a construção de variáveis preditivas (features) com base no comportamento passado. No entanto, devido a limitações de processamento, decidimos concentrar nossa análise em dados de apenas três meses.

In [ ]:
# Corrigindo valores negativos em 'total_secs' e mantendo valores positivos
logs_pd['total_secs'] = np.where(logs_pd['total_secs'] < 0, 0, logs_pd['total_secs'])

# Visualizando as primeiras linhas após o ajuste
print(logs_pd.head())

In [ ]:
# Gerando estatísticas descritivas para validar as alterações
print(logs_pd.describe())

### Lógica Temporal da Target e Features

A estratégia dos executivos é prever a saída da carteira de clientes, o que implica em algumas considerações importantes:

Uso de Dados Atuais: Durante a implementação do modelo, utilizaremos dados recentes de cadastro e de consumo, abrangendo desde os registros mais atuais até dados históricos, para prever a rentabilidade futura dos clientes.
Avaliação da Performance em M+1: O objetivo mencionado no texto é avaliar a performance no mês seguinte (M+1). Isso sugere que buscamos prever a rentabilidade dos clientes no mês subsequente.
Validação Out-of-Time (OOT): Para a validação da performance do modelo, propomos realizar um deslocamento (shift) de 1 mês à frente para o conjunto de teste. Essa abordagem permitirá avaliar a eficácia do modelo na previsão da rentabilidade futura de maneira robusta.

# B. Análise Exploratória
Avaliação de Volumetria

Modelo Atualizado: Para garantir que nosso modelo esteja alinhado com o cenário atual da companhia, utilizaremos a target referente ao mês de 201612 e avaliaremos a performance no mês de 201701.
1.1. É importante observar que os dados de cadastro não estão disponíveis a partir de 201701. Portanto, descartaremos os dados de consumo e pagamento referentes a 201702.

1.2. Os dados de pagamento nem sempre são registrados mensalmente, uma vez que alguns clientes optam pelo pagamento anual. Para resolver esse problema, filtraremos a última entrada de cada cliente na tabela de pagamentos, assegurando que tenhamos a informação mais recente disponível.

1.3. Identificamos também a presença de clientes sem assinatura que estão ouvindo música. Precisamos abordar esses casos de forma adequada. Uma questão a considerar é se o preço para esses clientes deve ser considerado como 0.

In [ ]:
# Corrigindo o cálculo de min e max para `msno` na base members_pd
print(members_pd.groupby("safra")["msno"].min(), members_pd.groupby("safra")["msno"].max())

In [ ]:
# Corrigindo o cálculo de min e max para `msno` na base logs_pd
print(logs_pd.groupby("safra")["msno"].min(), logs_pd.groupby("safra")["msno"].max())

In [ ]:
# Corrigindo o cálculo de min e max para `msno` na base transactions_pd
print(transactions_pd.groupby("safra")["msno"].min(), transactions_pd.groupby("safra")["msno"].max())

# Construção da Master de Treino
Compreendendo melhor as informações disponíveis, a volumetria e a distribuição dos dados, utilizaremos um histórico de 3 meses do cliente para prever a possibilidade de churn nos 3 meses seguintes.

Para isso, é essencial construir a master de treino e realizar ajustes necessários de feature engineering. Essa etapa incluirá a agregação de dados relevantes, a criação de variáveis preditivas e a aplicação de transformações adequadas nas variáveis existentes, a fim de melhorar a capacidade do modelo de identificar padrões que indiquem a evasão de clientes.

In [ ]:
import pandas as pd

# Lista das safras que serão utilizadas
safras = ['201612', '201609', '201608', '201607']  # Últimos 3 meses

# Inicializando uma lista para armazenar as tabelas unidas
all_join_tables = []

for safra in safras:
    # Filtrando os dados para a safra atual
    transactions_pd_target_full = transactions_pd[transactions_pd['safra'] <= int(safra)].copy()
    transactions_pd_train = transactions_pd[transactions_pd['safra'] < int(safra)].copy()

    # Selecionando a última entrada de cada cliente na tabela de transações
    transactions_pd_train['rownum'] = transactions_pd_train.sort_values(by=['safra', 'msno'], ascending=False)\
        .groupby('msno', sort=False).cumcount().add(1)
    transactions_pd_date = transactions_pd_train[transactions_pd_train['rownum'] == 1].drop('rownum', axis=1)

    transactions_pd_target_full['rownum'] = transactions_pd_target_full.sort_values(by=['safra', 'msno'], ascending=False)\
        .groupby('msno', sort=False).cumcount().add(1)
    transactions_pd_target = transactions_pd_target_full[transactions_pd_target_full['rownum'] == 1].drop('rownum', axis=1)

    # Filtrando logs para a safra atual e os meses anteriores
    members_pd_date = members_pd[members_pd['safra'] == safra]
    logs_pd_date_target = logs_pd[logs_pd['safra'] == int(safra)]
    logs_pd_date_3 = logs_pd[logs_pd['safra'] == int(safra) - 1]
    logs_pd_date_2 = logs_pd[logs_pd['safra'] == int(safra) - 2]
    logs_pd_date_1 = logs_pd[logs_pd['safra'] == int(safra) - 3]

    # Filtrando apenas clientes que ouviram música ou pagaram fatura
    join_tables_target = (members_pd_date.set_index('msno')
                          .join(logs_pd_date_target.set_index('msno'), how='inner', rsuffix='_logs_target')
                          .join(logs_pd_date_3.set_index('msno'), how='inner', rsuffix='_logs_m1')
                          .join(logs_pd_date_2.set_index('msno'), how='inner', rsuffix='_logs_m2')
                          .join(logs_pd_date_1.set_index('msno'), how='inner', rsuffix='_logs_m3')
                          .join(transactions_pd_target.set_index('msno'), how='inner', rsuffix='_transactions_target')
                          .join(transactions_pd_date.set_index('msno'), how='inner', rsuffix='_transactions_m1'))

    # Adicionando a tabela atual à lista de resultados
    all_join_tables.append(join_tables_target)

# Concatenando todas as tabelas unidas em um único DataFrame
join_tables = pd.concat(all_join_tables)

In [ ]:
# Visualizando as primeiras linhas da tabela unida
join_tables.head()

#### Novas variáveis

* tempo desde registro
* Audição (soma)
* Razão audição

In [ ]:
# Convertendo as colunas de data para datetime
join_tables.registration_init_time = pd.to_datetime(join_tables.registration_init_time, format='%Y%m%d', errors='coerce')
join_tables.safra = pd.to_datetime(join_tables.safra, format='%Y%m', errors='coerce')

# Calculando o tempo desde o registro
join_tables['registration_ate_hoje'] = (join_tables.registration_init_time.fillna(0).astype(int) - 
                                          join_tables.safra.fillna(0).astype(int))
join_tables['registration_ate_hoje'] = np.where(join_tables['registration_ate_hoje'] < 0, 0, join_tables['registration_ate_hoje'])


In [ ]:
# Corrigindo colunas de dias até expirar as transações
joiddn_tables['transaction_date_ate_expire_transactions_m1'] = np.where(join_tables['transaction_date_ate_expire_transactions_m1'] < 0, 0, join_tables['transaction_date_ate_expire_transactions_m1'])
join_tables['transaction_date_ate_expire'] = np.where(join_tables['transaction_date_ate_expire'] < 0, 0, join_tables['transaction_date_ate_expire'])


In [ ]:
# Soma dos segundos ouvidos nos últimos meses
join_tables['total_secs_ult_meses'] = (join_tables.total_secs + 
                                        join_tables.total_secs_logs_m1 + 
                                        join_tables.total_secs_logs_m2 + 
                                        join_tables.total_secs_logs_m3)

# Razão de audições
join_tables['razao_secs_ult_mes'] = join_tables.total_secs / join_tables.total_secs_logs_m1
join_tables['razao_secs_ult_3_mes'] = join_tables.total_secs / (join_tables.total_secs_logs_m1 + 
                                                               join_tables.total_secs + 
                                                               join_tables.total_secs_logs_m2)


In [ ]:
# Visualizar os resultados
print(join_tables.head())

# D. Construção da Target Final
Na etapa final, definiremos a lógica para a construção da variável target que irá indicar o churn dos clientes. Consideraremos um cliente como churn se ele atender a pelo menos uma das seguintes condições:

Cliente Inativo: O cliente não está ativo em sua assinatura.
Cliente Cancelado: O cliente optou por cancelar sua assinatura.
Premissa de Expiração: A data da última atividade do cliente deve ser menor ou igual à data de expiração de sua assinatura.
Essas definições nos permitirão criar uma target clara e objetiva, que servirá como base para o treinamento do modelo de previsão de churn.

In [ ]:
import numpy as np

# Definindo a variável target 'is_churn' conforme as condições de churn
join_tables['is_churn'] = np.where(
    (join_tables['is_ativo'] == '0') |  # Cliente não está ativo
    (join_tables['is_ativo'] != 1) |    # Cliente não está ativo (versão numérica)
    (join_tables['is_cancel'] == '1') | # Cliente cancelou o plano
    (join_tables['is_cancel_transactions_m1'] == '1'), # Cancelamento no mês anterior
    1, # Indica churn
    0  # Indica não churn
)


In [ ]:
# Verificando a distribuição e estatísticas dos dados por 'is_churn'
print(join_tables.groupby('is_churn').describe())


In [ ]:
# Checando valores nulos para entender a qualidade dos dados
print(join_tables.isna().sum())


Excluindo variaveis que formaram a target e podem gerar leakeage

In [ ]:
# Removendo as variáveis que podem gerar leakage
join_tables.drop(['is_cancel_transactions_m1', 'is_cancel', 'is_ativo'], axis=1, inplace=True)


In [ ]:
# Verificando se as colunas foram removidas corretamente
print("Colunas atuais:", join_tables.columns)

In [ ]:
# Calculando a taxa de churn
churn_rate = sum(join_tables['is_churn']) / len(join_tables['is_churn'])
print("Taxa de churn:", churn_rate)

dd

Temos 9% dos clientes em churn, será necessário estratificar

### Dropando variável de genero e data

In [ ]:
join_tables.drop('gender',axis =1, inplace= True)

In [ ]:
join_tables.drop(['registration_init_time', 'safra_logs_target', 'registered_via', 'safra_logs_m1', 'safra_logs_m2','transaction_date_transactions_m1', 'safra_transactions_m1',
                 'safra_logs_m3', 'transaction_date', 'membership_expire_date', 'safra_transactions_target', 'membership_expire_date_transactions_m1'],axis =1, inplace= True )

# E. Amostragem
### Explicação:
- A função `calcular_tamanho_amostra()` é usada para determinar o tamanho mínimo da amostra necessário com base no nível de confiança, proporção estimada e margem de erro.
- A função `gerar_amostra_dataset()` gera uma amostra do dataset usando o tamanho de amostra calculado. Para garantir que não tentamos amostrar mais do que o número total de observações no dataset, usamos `min(tamanho_amostra, len(dataset))`.
- O `random_state=42` garante que a amostragem seja reproduzível.
Esse código retorna a amostra do dataset com o número de confiança especificado.

In [ ]:
# Garantindo que as colunas de valor pago estejam no formato inteiro
join_tables['actual_amount_paid'] = join_tables['actual_amount_paid'].astype(int)
join_tables['actual_amount_paid_transactions_m1'] = join_tables['actual_amount_paid_transactions_m1'].astype(int)


In [ ]:
import math
import pandas as pd

# Função para calcular o tamanho mínimo da amostra
def calcular_tamanho_amostra(Z, p, E):
    n = (Z**2 * p * (1 - p)) / (E**2)
    return math.ceil(n)

# Função para gerar a amostra do dataset
def gerar_amostra_dataset(dataset, Z, p, E):
    # Calcular o tamanho da amostra
    tamanho_amostra = calcular_tamanho_amostra(Z, p, E)
    # Garantir que o tamanho da amostra não seja maior que o dataset original
    tamanho_amostra = min(tamanho_amostra, len(dataset))
    # Gerar a amostra
    amostra = dataset.sample(n=tamanho_amostra, random_state=42)
    return amostra

# Parâmetros de confiança
Z = 1.96  # 95% de nível de confiança
p = 0.5   # Proporção populacional estimada
E = 0.05  # Margem de erro de 5%

# Gerar a amostra
amostra = gerar_amostra_dataset(join_tables, Z, p, E)

In [ ]:
# Verificando o tamanho da amostra gerada
amostra.head()

In [ ]:
# Verificando o tamanho da amostra gerada
print("Tamanho da amostra:", len(amostra))


# G. Testes de multicolinearidade
- Spearman: Numéricas
- MCC: Categoricas

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics import matthews_corrcoef
from itertools import combinations

# Função para calcular a correlação de Spearman para variáveis numéricas
def spearman_corr(df, threshold=0.9):
    corr_matrix = df.corr(method='spearman')
    drop_cols = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                drop_cols.add(corr_matrix.columns[i])
    return drop_cols

# Função para calcular o coeficiente MCC entre variáveis categóricas
def mcc_corr(df, threshold=0.9):
    drop_cols = set()
    for col1, col2 in combinations(df.columns, 2):
        try:
            mcc_value = matthews_corrcoef(df[col1], df[col2])
            if abs(mcc_value) > threshold:
                drop_cols.add(col1)  # Remover col1 se altamente correlacionado
        except ValueError:
            continue  # Ignora colunas com valores únicos
    return drop_cols

# Função principal para identificar e remover colinearidades
def remove_colinearity(df, num_threshold=0.9, cat_threshold=0.9):
    # Separar variáveis numéricas e categóricas
    num_df = df.select_dtypes(include=[np.number])
    cat_df = df.select_dtypes(include=[object, 'category'])
    # Verificar colinearidade em variáveis numéricas
    num_cols_to_drop = spearman_corr(num_df, num_threshold)
    # Verificar colinearidade em variáveis categóricas
    cat_cols_to_drop = mcc_corr(cat_df, cat_threshold)
    # Remover colunas altamente correlacionadas
    df_cleaned = df.drop(columns=list(num_cols_to_drop) + list(cat_cols_to_drop))
    return df_cleaned


In [ ]:
#Aplicando ao dataset amostra
dataset_cleaned = remove_colinearity(amostra, num_threshold=0.9, cat_threshold=0.9)


In [ ]:
# Atualizando join_tables para refletir as colunas selecionadas
join_tables = join_tables[dataset_cleaned.columns]

In [ ]:
# Visualizando o resultado
join_tables.head()

# E. Divisão de treino e teste
Com as variaveis importantes, dividiremos nossa base entre treino e teste. Como estratégia para qualdiade, dividiremos nossa base entre treino, teste e validação. Além disso,usaremos a proporção 60%/40%.

Anterior à divisão excluiremos a variável de *genero* e faremos check de colinariedade

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Preparando os dados
join_tables = join_tables.reset_index(drop=True)

# Excluindo a variável 'gender' e outras colunas irrelevantes, verificando também tipos de dados
join_tables = join_tables.drop(columns=['gender', 'msno', 'safra'], errors='ignore')
join_tables['payment_plan_days_transactions_m1'] = join_tables['payment_plan_days_transactions_m1'].astype(int)
join_tables['plan_list_price_transactions_m1'] = join_tables['plan_list_price_transactions_m1'].astype(int)
join_tables['is_auto_renew_transactions_m1'] = join_tables['is_auto_renew_transactions_m1'].astype(int)
join_tables['payment_method_id_transactions_m1'] = join_tables['payment_method_id_transactions_m1'].astype(int)
join_tables['city'] = join_tables['city'].astype(int)
join_tables['bd'] = join_tables['bd'].astype(int)

# Verificação de valores nulos
missing_values = join_tables.isna().sum().sum()
print(f'Total de valores nulos: {missing_values}')

O erro indica que há valores nulos

In [ ]:
# Preenchendo valores nulos com 0 nas colunas que serão convertidas para int
join_tables['payment_plan_days_transactions_m1'].fillna(0, inplace=True)
join_tables['plan_list_price_transactions_m1'].fillna(0, inplace=True)
join_tables['is_auto_renew_transactions_m1'].fillna(0, inplace=True)
join_tables['payment_method_id_transactions_m1'].fillna(0, inplace=True)
join_tables['city'].fillna(0, inplace=True)
join_tables['bd'].fillna(0, inplace=True)

# Convertendo colunas para inteiro
join_tables['payment_plan_days_transactions_m1'] = join_tables['payment_plan_days_transactions_m1'].astype(int)
join_tables['plan_list_price_transactions_m1'] = join_tables['plan_list_price_transactions_m1'].astype(int)
join_tables['is_auto_renew_transactions_m1'] = join_tables['is_auto_renew_transactions_m1'].astype(int)
join_tables['payment_method_id_transactions_m1'] = join_tables['payment_method_id_transactions_m1'].astype(int)
join_tables['city'] = join_tables['city'].astype(int)
join_tables['bd'] = join_tables['bd'].astype(int)

# Verificação de valores nulos
missing_values = join_tables.isna().sum().sum()
print(f'Total de valores nulos após preenchimento: {missing_values}')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Preparando os dados
join_tables = join_tables.reset_index(drop=True)

# Excluindo a variável 'gender' e outras colunas irrelevantes, verificando também tipos de dados
join_tables = join_tables.drop(columns=['gender', 'msno', 'safra'], errors='ignore')
join_tables['payment_plan_days_transactions_m1'] = join_tables['payment_plan_days_transactions_m1'].astype(int)
join_tables['plan_list_price_transactions_m1'] = join_tables['plan_list_price_transactions_m1'].astype(int)
join_tables['is_auto_renew_transactions_m1'] = join_tables['is_auto_renew_transactions_m1'].astype(int)
join_tables['payment_method_id_transactions_m1'] = join_tables['payment_method_id_transactions_m1'].astype(int)
join_tables['city'] = join_tables['city'].astype(int)
join_tables['bd'] = join_tables['bd'].astype(int)

# Verificação de valores nulos
missing_values = join_tables.isna().sum().sum()
print(f'Total de valores nulos: {missing_values}')

In [ ]:
# Separação de features (X) e target (y)
X = join_tables.drop(columns=['is_churn'])
y = join_tables['is_churn']

In [ ]:
# Dividindo os dados em treino (60%) e temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Dividindo temp (40%) em validação (40% de temp) e teste (60% de temp)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6, random_state=42, stratify=y_temp)

In [ ]:
# Exibindo as dimensões dos conjuntos de dados
print(f"Dimensões do conjunto de treino: {X_train.shape}")

In [ ]:
print(f"Dimensões do conjunto de validação: {X_val.shape}")

In [ ]:
print(f"Dimensões do conjunto de teste: {X_test.shape}")

In [ ]:
# Conferindo as primeiras linhas do conjunto de treino
X_train.head()

# F. Feature Engineering

Etapa necessária pré treino do modelo, iremos fazer os casts necessários, ajuste de dummies e outros.

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1. Identificar variáveis categóricas e numéricas
categorical_cols = ['payment_method_id_transactions_m1', 'city']  # Exemplo de variáveis categóricas, ajuste conforme necessário
numerical_cols = [col for col in X_train.columns if col not in categorical_cols]

# 2. Converting categorical variables to category type
for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')
    X_test[col] = X_test[col].astype('category')

# 3. One-hot encoding para variáveis categóricas
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_val = pd.get_dummies(X_val, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Certificar que as colunas de treino, validação e teste estão alinhadas após o encoding
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 4. Padronização das variáveis numéricas
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [ ]:
# Visualizando as dimensões dos conjuntos após o processamento
print("Dimensões do conjunto de treino:", X_train.shape)

In [ ]:
print("Dimensões do conjunto de validação:", X_val.shape)

In [ ]:
print("Dimensões do conjunto de teste:", X_test.shape)

# G.Feature Selection

É interessante separar o que realmente influencia ou não no churn, para tanto, devido a termos uma base mixada (variaveis categoricas e numericas) utilizaremos uma abordagem baseada em ensemble.

LightGBM é extremamente eficiente para data frames grandes.
Como o Random Forest, ele também fornece importâncias de variáveis, mas é otimizado para desempenho em datasets grandes e com alto número de features.
Ele pode lidar bem com variáveis categóricas, e geralmente é mais rápido que Random Forest devido à sua implementação baseada em histogramas. Além disso, atende bem problemas desbalanceados como  o nosso,

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Definindo o modelo LightGBM para feature selection
lgb_model = lgb.LGBMClassifier(random_state=42, n_estimators=500)

# Treinando o modelo com todas as features
lgb_model.fit(X_train, y_train)

# Extraindo as importâncias das features
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_model.feature_importances_
})

# Ordenando por importância e selecionando as principais variáveis
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Definindo um número de features a serem mantidas
top_k = 20
top_features = feature_importances.head(top_k)['feature'].values

# Selecionando as features mais importantes no conjunto de dados
X_train_selected = X_train[top_features]
X_val_selected = X_val[top_features]
X_test_selected = X_test[top_features]

In [ ]:
# Verificação das dimensões após a seleção
print("Dimensões do conjunto de treino com features selecionadas:", X_train_selected.shape)

In [ ]:
print("Dimensões do conjunto de validação com features selecionadas:", X_val_selected.shape)

In [ ]:
print("Dimensões do conjunto de teste com features selecionadas:", X_test_selected.shape)

Com as variáveis definidas vamos iniciar o treinamento do algoritmo

# H. Treinando o algoritmo

Para o treinamento optamos por testar diferentes classes, cujos principais pontos positivos e negativos estão descritos abaixo:


### 1. **LightGBM**
  - **Por que usar:**  Muito rápido e eficiente em termos de uso de memória, especialmente em conjuntos de dados grandes, permite ajuste fino de muitos parâmetros, como a profundidade da árvore e o número de folhas, para melhorar o desempenho e lida bem com conjuntos de dados esparsos, o que é comum em problemas com muitas variáveis categóricas.
  - **Por que não usar LightGBM:** tem muitoshiperparâmetros que precisam ser ajustados corretamente, o que pode ser complexo e demorado e como outros algoritmos de gradient boosting, pode ser suscetível a overfitting, especialmente em conjuntos de dados pequenos ou mal balanceados.

### 2. **XGBoost (XGBClassifier)**
  - **Por que usar:** É  uma técnica de boosting extremamente eficiente e poderosa que geralmente supera muitos outros algoritmos em termos de performance em problemas de classificação. Ele é bom em capturar interações complexas e pode lidar com dados desbalanceados, que são comuns em problemas de churn.
  - **Por que não usar:** Similar ao Random Forest, XGBoost pode ser caro em termos computacionais. Além disso, requer um ajuste de hiperparâmetros mais cuidadoso para evitar overfitting, o que pode aumentar o tempo de desenvolvimento.

### 3. **Random Forest**
  - **Por que usar:** Robustex cobtra Overfiting, lida bem com dados faltantes
  - **Por que não usar:** Propensas a Bias

Aqui está um código que implementa os ajustes para `scale_pos_weight`, `sample_weight`, e o ajuste do threshold de decisão no XGBoost para lidar com dados desbalanceados:
### Explicação:
1. **Ajuste do `scale_pos_weight`** (ponto 2): Calculamos a razão entre as amostras negativas e positivas no conjunto de treino para balancear o impacto das classes.
2. **Uso de `sample_weight`** (ponto 4): Aplicamos pesos diferentes às amostras, dando mais importância às da classe minoritária.
3. **Threshold tuning** (ponto 5): Ao invés de usar o threshold padrão de 0.5 para a probabilidade de previsão, ajustamos para 0.3 para captar mais positivos.
Esse código pode ser adaptado ao seu problema específico, ajustando os parâmetros conforme necessário.

In [ ]:
# Calculando scale_pos_weight
num_pos = np.sum(y_train == 1)  # Número de positivos
num_neg = np.sum(y_train == 0)  # Número de negativos
scale_pos_weight = num_neg / num_pos

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
# Application of all Machine Learning methods
MLA = [
    LogisticRegression(class_weight='balanced',C=1),

    # LightGBM
    LGBMClassifier(scale_pos_weight=scale_pos_weight,  # Ajuste para classes desbalanceadas
   use_label_encoder=True),

    #Ensemble Methods
    XGBClassifier( scale_pos_weight=scale_pos_weight,  # Ajuste para classes desbalanceadas
   use_label_encoder=True),

    #Random Forest
    RandomForestClassifier(max_depth=2,class_weight = 'balanced')

    ]

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, roc_curve, auc, f1_score
from sklearn.model_selection import train_test_split, KFold
from sklearn import linear_model, ensemble, tree, naive_bayes
from sklearn.ensemble import RandomForestClassifier
import xgboost
import numpy as np


# Inicializando as colunas do DataFrame para comparação
MLA_columns = ['MLA used', 'Train Accuracy', 'Validation Accuracy', 'Test Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']
MLA_compare = pd.DataFrame(columns=MLA_columns)

# Índice para linhas do DataFrame
row_index = 0

# Loop para cada algoritmo na lista MLA
for alg in MLA:
    print(f"Evaluando {alg.__class__.__name__}...")

    # Validação Cruzada usando o conjunto de treinamento
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    val_accuracies = []
    val_precisions = []
    val_recalls = []
    val_f1_scores = []
    val_auc_scores = []

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        # Treinando o modelo
        alg.fit(X_train_fold, y_train_fold)

        # Predições e métricas no conjunto de validação
        y_val_pred = alg.predict(X_val_fold)
        y_val_pred_proba = alg.predict_proba(X_val_fold)[:, 1] if hasattr(alg, 'predict_proba') else y_val_pred

        val_accuracies.append(alg.score(X_val_fold, y_val_fold))
        val_precisions.append(precision_score(y_val_fold, y_val_pred))
        val_recalls.append(recall_score(y_val_fold, y_val_pred))
        val_f1_scores.append(f1_score(y_val_fold, y_val_pred))
        fp, tp, _ = roc_curve(y_val_fold, y_val_pred_proba)
        val_auc_scores.append(auc(fp, tp))

    # Calculando métricas médias
    val_accuracy = round(np.mean(val_accuracies), 4)
    val_precision = round(np.mean(val_precisions), 4)
    val_recall = round(np.mean(val_recalls), 4)
    val_f1 = round(np.mean(val_f1_scores), 4)
    val_auc = round(np.mean(val_auc_scores), 4)

    # Treinamento final no conjunto de treino
    alg.fit(X_train, y_train)

    # Avaliação final no conjunto de teste
    y_test_pred = alg.predict(X_test)
    y_test_pred_proba = alg.predict_proba(X_test)[:, 1] if hasattr(alg, 'predict_proba') else y_test_pred

    test_accuracy = round(alg.score(X_test, y_test), 4)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    fp, tp, _ = roc_curve(y_test, y_test_pred_proba)
    auc_score = auc(fp, tp)

    # Armazenando métricas no DataFrame
    MLA_compare.loc[row_index] = [alg.__class__.__name__,
                                   round(alg.score(X_train, y_train), 4),
                                   val_accuracy,
                                   test_accuracy,
                                   precision,
                                   recall,
                                   f1,
                                   auc_score]

    row_index += 1

In [ ]:
MLA_compare.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)
MLA_compare

# I. Hiperparametrização

Escolhemos os parametros de redução de complexidade do modelo, regularização, subamostragem e aprendizado focando em melhorar assertividade do modelo preservando o trade-off entre viés e variância, abaixo detalho os parametros:

1) Redução de complexidade - profundidade das árvores, quanto mais rasa menos propensa a overfitar

2) Regularização L1 para esparsidade

3) Taxa de Aprendizado

4) Redução de fração de amostras na contrução da arvore

Para realizar a troca de `RandomizedSearchCV` por uma busca bayesiana, você pode utilizar a biblioteca `BayesSearchCV` do pacote `scikit-optimize`, que é uma alternativa eficiente para otimização de hiperparâmetros. O principal benefício da busca bayesiana é que ela explora o espaço de parâmetros de forma mais inteligente, ajustando as tentativas com base nos resultados anteriores.
Aqui está o código modificado para usar a busca bayesiana, e com a métrica de recall como critério de avaliação:

### Principais alterações:
1. **Biblioteca usada:** `BayesSearchCV` do `scikit-optimize`.
2. **Espaço de parâmetros:** A sintaxe de intervalos é ligeiramente diferente, com o uso de `(min, max)` e distribuições como `'log-uniform'`.
3. **Critério de avaliação:** O recall foi definido como métrica principal usando `make_scorer(recall_score)` e passado para o parâmetro `scoring`.
4. **Parâmetro `n_estimators`:** O intervalo foi ampliado para permitir uma busca mais ampla.
Essa abordagem permitirá uma busca mais eficiente, focando em otimizar o recall.

In [ ]:
import xgboost as xgb
from sklearn.metrics import make_scorer, recall_score
from skopt import BayesSearchCV
from xgboost import XGBClassifier

# Definir o modelo inicial
model = XGBClassifier(scale_pos_weight=scale_pos_weight, use_label_encoder=False)

# Parâmetros para Busca Bayesiana (extremamente simplificados)
param_dist = {
    'max_depth': (1, 2),  # Profundidade mínima
    'reg_alpha': (0.01, 0.1),  # Regularização leve
    'learning_rate': (0.01, 0.1),  # Taxa de aprendizado baixa
    'n_estimators': (5, 10),  # Valor muito baixo para árvores
    'subsample': (0.5, 1.0),  # Usar entre 50% a 100% dos dados
    'colsample_bytree': (0.5, 1.0)  # Usar entre 50% a 100% das colunas
}

# Scorer baseado em recall
recall = make_scorer(recall_score)

# Amostragem do conjunto de dados (1% dos dados)
X_train_sample = X_train.sample(frac=0.01, random_state=42)  # Usar 1% dos dados
y_train_sample = y_train.loc[X_train_sample.index]

# BayesSearchCV com validação cruzada (configurado para ser extremamente rápido)
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_dist,
    n_iter=1,  # Apenas uma iteração
    cv=2,  # Validação cruzada com 2 folds
    n_jobs=1,  # Usar apenas um núcleo para evitar sobrecarga
    verbose=0,  # Silenciar logs
    random_state=3,
    scoring=recall
)

# Aplicando a busca (sem avaliação)
bayes_search.fit(X_train_sample, y_train_sample)


In [ ]:
# Melhor modelo e parâmetros
best_model_bayes = bayes_search.best_estimator_
print(f"Best parameters (BayesSearchCV): {bayes_search.best_params_}")

In [ ]:
print(f"Best score (BayesSearchCV - Recall): {bayes_search.best_score_}")

In [ ]:
# Avaliação do modelo final
y_pred_bayes = best_model_bayes.predict(X_test)
y_pred_proba_bayes = best_model_bayes.predict_proba(X_test)[:, 1]

# Métricas
accuracy_bayes = recall_score(y_test, y_pred_bayes)
auc_bayes = roc_auc_score(y_test, y_pred_proba_bayes)
f1_bayes = f1_score(y_test, y_pred_bayes)

In [ ]:
print(f"Test recall (BayesSearchCV): {accuracy_bayes}")

In [ ]:
print(f"Test AUC (BayesSearchCV): {auc_bayes}")

In [ ]:
print(f"Test F1 Score (BayesSearchCV): {f1_bayes}")

Agora que descobrimos os melhores parametros para o algoritmo vamos resolver a previsão 3 meses a frente.

# J. Prevendo 3 meses a frente
Por fim iremos finalizar a previsão dos clientes que nao deram churn neste mês mas que poderiam dar no proximo ou em M2, M3 com os valores otimos da busca bayesiana. 

Criaremos também uma função de visualização para os resultados do modelo.

In [ ]:
join_tables['safra'] = pd.to_datetime(join_tables.safra, format='%Y%m', errors='coerce').dt.strftime('%Y%m')

In [ ]:
# Notebook 4
join_tables['churn_1'] = join_tables.groupby('msno')['is_churn'].shift(-1)
join_tables['churn_2'] = join_tables.groupby('msno')['is_churn'].shift(-2)
join_tables['churn_3'] = join_tables.groupby('msno')['is_churn'].shift(-3)

In [ ]:
# Previsão para M1
x1m = join_tables.dropna(subset=['churn_1']).filter(feature_importance_df)
y_1m = join_tables.dropna(subset=['churn_1'])['churn_1']

# Previsão para M2
x2m = join_tables.dropna(subset=['churn_2']).filter(feature_importance_df)
y_2m = join_tables.dropna(subset=['churn_2'])['churn_2']

# Previsão para M3
x3m = join_tables.dropna(subset=['churn_3']).filter(feature_importance_df)
y_3m = join_tables.dropna(subset=['churn_3'])['churn_3']


In [ ]:
join_tables.sort_values(by=['msno', 'safra'], ascending = False, inplace = True)

In [ ]:
# Treinamento e teste para previsão de 1 mês (M1)
X_train1, X_test1m, y_train_1m, y_test_1m = train_test_split(x1m, y_1m, test_size=0.3, random_state=42)
costumer_churn_prediction(xgc, X_train1, X_test1m, y_train_1m, y_test_1m, "features", threshold_plot=True)

# Treinamento e teste para previsão de 2 meses (M2)
X_train2, X_test2m, y_train_2m, y_test_2m = train_test_split(x2m, y_2m, test_size=0.3, random_state=42)
costumer_churn_prediction(xgc, X_train2, X_test2m, y_train_2m, y_test_2m, "features", threshold_plot=True)

# Treinamento e teste para previsão de 3 meses (M3)
X_train3, X_test3m, y_train_3m, y_test_3m = train_test_split(x3m, y_3m, test_size=0.3, random_state=42)
costumer_churn_prediction(xgc, X_train3, X_test3m, y_train_3m, y_test_3m, "features", threshold_plot=True)


In [ ]:
# Previsão e cálculo do churn em cada janela
for period, y_pred, y_test in zip(['1 mês', '2 meses', '3 meses'], 
                                  [y_pred_1m, y_pred_2m, y_pred_3m], 
                                  [y_test_1m, y_test_2m, y_test_3m]):
    total_churn = (y_pred == 1).sum()
    print(f'Percentual de clientes com churn em {period}: {total_churn / len(y_pred):.2%}')


In [ ]:
#Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import * # module math
import matplotlib.pyplot as plt # visualization
from PIL import Image
import seaborn as sns # visualization
import itertools
import io
import plotly.offline as py # visualization
py.init_notebook_mode(connected=True) # visualization
import plotly.graph_objs as go # visualization
from plotly.subplots import make_subplots
import plotly.figure_factory as ff # visualization
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import statsmodels.api as sm
from yellowbrick.classifier import DiscriminationThreshold

%matplotlib inline

In [ ]:
def costumer_churn_prediction(algorithm, training_x, testing_x, training_y, testing_y, cf, threshold_plot):
    #model
    algorithm.fit(training_x, training_y)
    predictions = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
        
    print('Algorithm:', type(algorithm).__name__)
    print("\nClassification report:\n", classification_report(testing_y, predictions))
    print("Accuracy Score:", accuracy_score(testing_y, predictions))
    
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y, predictions)
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y, predictions) 
    print("Area under curve:", model_roc_auc,"\n")
    
    fpr, tpr, thresholds = roc_curve(testing_y, probabilities[:,1])
     
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix,
                        x = ["Not churn", "Churn"],
                        y = ["Not churn", "Churn"],
                        showscale = False, colorscale = "Picnic",
                        name = "Confusion matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr, y = tpr,
                        name = "Roc: " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'), width = 2))
    trace3 = go.Scatter(x = [0,1], y = [0,1],
                        line = dict(color = ('rgb(205, 12, 24)'), width = 2,
                        dash = 'dot'))
    
    if cf in ['coefficients', 'features']:
        if cf == 'coefficients':
            coefficients = pd.DataFrame(algorithm.coef_.ravel())
        elif cf == 'features':
            coefficients = pd.DataFrame(algorithm.feature_importances_)
        
        column_df = pd.DataFrame(training_x.columns.tolist())
        coef_sumry = (pd.merge(coefficients, column_df, left_index=True, 
                               right_index=True, how="left"))
        coef_sumry.columns = ["coefficients", "features"]
        coef_sumry = coef_sumry.sort_values(by = "coefficients", ascending=False)
        
        #plot coeffs
        trace4 = go.Bar(x = coef_sumry["features"], y = coef_sumry["coefficients"], 
                        name = "coefficients",
                        marker = dict(color = coef_sumry["coefficients"],
                                      colorscale = "Picnic",
                                      line = dict(width = .6, color = "black")
                                     )
                       )
        #subplots
        fig = make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                                subplot_titles=('Confusion matrix',
                                                'Receiver operating characteristic',
                                                'Feature importances')
                           )  
        fig.append_trace(trace1,1,1)
        fig.append_trace(trace2,1,2)
        fig.append_trace(trace3,1,2)
        fig.append_trace(trace4,2,1)
        fig['layout'].update(showlegend=False, title="Model performance",
                             autosize=False, height = 900, width = 800,
                             plot_bgcolor = 'rgba(240,240,240, 0.95)',
                             paper_bgcolor = 'rgba(240,240,240, 0.95)',
                             margin = dict(b = 195))
        fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
        fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
        fig["layout"]["xaxis3"].update(dict(showgrid = True, tickfont = dict(size = 10), tickangle = 90))
        
    elif cf == 'None':
        #subplots
        fig = make_subplots(rows=1, cols=2,
                            subplot_titles=('Confusion matrix',
                                            'Receiver operating characteristic')
                           )
        fig.append_trace(trace1,1,1)
        fig.append_trace(trace2,1,2)
        fig.append_trace(trace3,1,2)
        fig['layout'].update(showlegend=False, title="Model performance",
                         autosize=False, height = 500, width = 800,
                         plot_bgcolor = 'rgba(240,240,240,0.95)',
                         paper_bgcolor = 'rgba(240,240,240,0.95)',
                         margin = dict(b = 195))
        fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
        fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))  
        
    py.iplot(fig)
    
    if threshold_plot == True: 
        visualizer = DiscriminationThreshold(algorithm)
        visualizer.fit(training_x,training_y)
        visualizer.poof()

In [ ]:
# Separar variáveis explicativas e alvo para previsão de churn no próximo mês

# Dividir os dados em conjuntos de treinamento e teste
X_train1, X_test1m, y_train_1m, y_test_1m = train_test_split(x1m, y_1m, test_size=0.3, random_state=42)


from xgboost import XGBClassifier

xgc = XGBClassifier(scale_pos_weight=scale_pos_weight,  # Ajuste para classes desbalanceadas
   use_label_encoder=False, reg_lambda =  0.1, reg_alpha = 0.644, n_estimators = 50,
                         max_depth = 2, learning_rate = 0.11)

costumer_churn_prediction(xgc, X_train1, X_test1m, y_train_1m, y_test_1m, "features", threshold_plot=True)

In [ ]:
print('Percentual de clientes com churn:',(total_churn_1m)/(len(y_pred_1m)))

## **Conclusão:**
Conseguimos identificar 6% clientes com possível  churn nos proximos três meses.
Entendendo que X% ficam ativos pós ação, conseguiremos reter X%.

É importante passar a lista destes clientes para que a area de negocio entre em contato tentando fortalecer e restabelecer o relacionamento com o objetivo de reduzir o provável churn.

Para finalizar, iremos realizar a analise não supervisionada, agrupar nossos clientes e tentar buscar padrões de churn e comportamento.

# 2. Analise de Perfil de clientes

Aqui o objetivo é entender mais sobre o padrão comportamente dos nossos clientes através da analise supervisionada.
Para tanto, utilizaremos duas técnicas diferentes: de centroides e de hierarquização, para centoide direcionamos ao  K-means e Clusterização hierarquica.

Seguiremos um passo a passo que consiste em:


1. Remoção de outliers e PCA

2. Numero de K otimo - Elbow

3. Aplicar a clusterização usando K-means

4. Aplicar a clusterização hierarquica

5. Comparar os resultados - Rand Index e coeifciente de silueta

# A.Remoção de outliers e Redução de Dimensionalidade

Redução de Outliers com Quantis

In [ ]:
def out_data(df):
    for col in df.columns:
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.01,0.99]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

final_df=out_data(join_tables)

Devido ao tamanho da base aplicaremos amostragem para categorização. Utilizaremos 30% aleatoriamente definido para a amostra, **amostragem aleatoria simples com reposição**

In [ ]:
sample_1 = final_df[:10000]

# B. Número ideal de k

As principais metodologias de definição de K são Cotovelo (elbow) e silhueta. Para definir com extatidão qual seguir vamos fazer  um comparação entre elas, com seus prós e contras:
#### Método do Cotovelo (Elbow)
#### Descrição:
O método do cotovelo consiste em calcular a soma das distâncias quadráticas dentro dos clusters (inertia ou WCSS - Within-Cluster Sum of Squares) para diferentes valores de \( K \). O valor ideal de \( K \) é identificado onde há uma diminuição acentuada na curva, formando um "cotovelo".

#### Prós:
- Simplicidade: Fácil de implementar e interpretar visualmente.
- Intuitivo: A curva do cotovelo fornece uma maneira visualmente clara de selecionar \( K \).
#### Contras:
- Subjetivo: A identificação do "cotovelo" pode ser ambígua, especialmente em datasets onde a curva é suave.
- Escalabilidade: Pode ser computacionalmente caro para grandes datasets, pois requer múltiplas execuções do algoritmo de clustering.

A técnica de normalização escolhidade é o minimos máximos

**Devido ao alto processamento computacional seguiremos com o metodo do cotovelo.**

In [ ]:
# Identificar colunas numéricas e categóricas
num_cols = sample_1.drop(['msno', 'safra', 'churn_1','is_churn', 'plan_list_price','payment_plan_days','is_auto_renew','bd',
 'registration_ate_hoje','transaction_date_ate_expire_transactions_m1','transaction_date_ate_expire',
                          'payment_method_id','payment_plan_days',
 ], axis =1).select_dtypes(include=np.number).columns.tolist()

# Identificar colunas numéricas e categóricas
cat_cols = sample_1.drop(['msno', 'safra', 'churn_1','is_churn', 'plan_list_price','payment_plan_days','is_auto_renew','bd',
 'registration_ate_hoje','transaction_date_ate_expire_transactions_m1','transaction_date_ate_expire',
                          'payment_method_id','payment_plan_days',
 ], axis =1).select_dtypes(include=np.object).columns.tolist()

In [ ]:
#Iniciando parametros do kmeans
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"random_state": 1,
}

# Normalizar os dados numéricos
scaler = MinMaxScaler()
scaler = scaler.fit(sample_1[num_cols])
sample_1[num_cols] = scaler.transform(sample_1[num_cols])
#print(scaled)


#create list to hold SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(sample_1[num_cols])
    sse.append(kmeans.inertia_)

#visualize results
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters nobs ")
plt.ylabel("SSE")
plt.show()

# C. Redução de dimensionalidade (PCA)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

pca = PCA(2)
data_pca = pca.fit_transform(sample_1[num_cols])

In [ ]:
plt.scatter(data_pca[:, 0], data_pca[:, 1],
            c=sample_1.is_churn, edgecolor='none', alpha=0.5)
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

# D. Clusterização
Para o agrupamento utilizamos duas técnicas: k-means e GMM

### Clusterização usando K-means

In [ ]:
# Aplicando K-Means
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans.fit_predict(data_pca)
kmeans_silhouette = silhouette_score(data_pca, kmeans_labels)

# salvando na base
sample_1['kmeans_cluster'] = kmeans_labels


# Métrica de qualidade
print(f'Coeficiente de Silhueta - KMeans: {kmeans_silhouette:.4f}')

In [ ]:
# for inverse transformation
sample_1[num_cols] = scaler.inverse_transform(sample_1[num_cols])
sample_1.groupby('kmeans_cluster').describe()

### Clusterização usando GMM

Para garantir que os dados estejam mais próximos de uma distribuição normal, você pode aplicar uma transformação logarítmica ou a transformação Box-Cox (que é mais flexível) antes da normalização. A seguir, o código atualizado com a transformação Box-Cox (caso os dados sejam estritamente positivos) antes da normalização.

### Explicações:
1. **PowerTransformer (Box-Cox)**: Essa transformação aproxima a normalidade para variáveis positivas, ajustando distribuições assimétricas ou com caudas longas. Use `method='box-cox'` para a transformação Box-Cox.
  - Nota: O Box-Cox requer que os dados sejam **estritamente positivos**.
  - Caso você tenha variáveis que incluem valores zero ou negativos, pode usar `method='yeo-johnson'`, que também é uma transformação de potência, mas não tem essa restrição.
2. **StandardScaler**: Normaliza as variáveis numéricas transformadas para média 0 e desvio padrão 1.
3. **OneHotEncoder**: Codifica as variáveis categóricas em formato binário (dummy variables).
### Resultados:
Esse pipeline agora trata adequadamente os dados numéricos, aplicando uma transformação que aproxima uma distribuição normal e em seguida normaliza os dados para que o **GMM** funcione melhor, especialmente em situações onde as variáveis não são originalmente gaussianas.
Se você tiver dúvidas sobre o uso do Box-Cox ou a necessidade de outro tipo de transformação, é só avisar!

In [ ]:
x.describe()

In [ ]:
sample_1.describe()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture

# Criar transformações
transformacoes = ColumnTransformer(transformers=[
   ('num', Pipeline(steps=[
       #('boxcox', PowerTransformer(method='box-cox')),  # Transformação Box-Cox para aproximar normalidade
       ('scaler', StandardScaler())  # Normalização
   ]), num_cols),
   ('cat', OneHotEncoder(), cat_cols)
])
# Criar pipeline com pré-processamento e GMM
pipeline = Pipeline(steps=[
   ('pre_processamento', transformacoes),
   ('gmm', GaussianMixture(n_components=3, random_state=42))
])
# Ajustar modelo
pipeline.fit(sample_1)
# Prever clusters
clusters = pipeline['gmm'].predict(pipeline['pre_processamento'].transform(sample_1))
print("Clusters:", clusters)

## **Conclusão:**
Após a escolha de 4 grupos através da distãncia intra clusters de elbow,clusterizamos com o algoritmo kmeans e abordagem de distancia euclidiana. Os grupos apresentaram distinção entre quem deu churn ou não, no mes target e outros. Outro  insight relevante foi a distinção de audição entre os grupos, e uma relação observada entre churn e a propria audição dos grupos.